In [1]:
import sys
import os
import pandas as pd
import numpy as np

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [130]:
import torch
from torch import nn
from torch import functional as F

batch_size = 32
block_size = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
DATA_PATH = os.path.join(PROJECT_ROOT, "data")
PATH_RATINGS = os.path.join(DATA_PATH, "raw_data", "kaggle_second_sem", "books_rating.csv")
PATH_BOOKS = os.path.join(DATA_PATH, "raw_data", "kaggle_second_sem", "books_data.csv")
PATH_EMBDS = os.path.join(DATA_PATH, "embeddings", "expanded_embds_ss.npy")

df_books = pd.read_csv(PATH_BOOKS)
df_ratings = pd.read_csv(PATH_RATINGS)
book_embds = np.load(PATH_EMBDS, allow_pickle=True)

# Book embds has shape like
#(Title, (author, 1), (categories, 1), (publisher, 1), (description_embd, 384))

In [ ]:
ratings_seqs = df_ratings.groupby("User_id")["Title"].apply(lambda x: list(set(x.tolist()))).loc[lambda x: x.str.len() > 5].reset_index()
train = ratings_seqs["Title"].iloc[:30000]
test = ratings_seqs["Title"].iloc[30000:]

In [140]:
torch.randint(2, (3, 3))

# torch.randint(len(train[1636]) - block_size, (1,))
torch.randint(len(train[20551]) - block_size, (1,))

tensor([11])

In [164]:
book_embds_dict = {row[0]: np.array(row[1:], dtype=np.float32) for row in book_embds}

def encode(book_title):
    return book_embds_dict[book_title]
def encode_seq(seq):
    return [book_embds_dict[title] for title in seq]


In [161]:
def encode_seq(seq):
    for title in seq:
        print(title)
    return [book_embds_dict[title] for title in seq]

In [ ]:
def get_batch(split):
    data = train if split == 'train' else test
    ix = torch.randint(len(train), (batch_size,))

    x = []
    y = []
    for i in ix:
        i = int(i)       
        j = torch.randint(len(data[i]) - block_size, (1,))
        x.append(encode_seq(data[i][j:j + block_size]))
        y.append(encode_seq(data[i][j + 1:j + block_size + 1]))

    x = torch.tensor(x)
    y = torch.tensor(y)
    x, y = x.to(device), y.to(device)
    return x, y

In [166]:
get_batch('train')

C:\Users\Ivans\AppData\Local\Temp\ipykernel_10036\2990964281.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  x = torch.tensor(x)


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [36]:
book_embds_dict["Open Secret: Lost... But Not Forgotten (Harlequin Superromance No 1332)"]

array([ 4.00000000e+00,  3.00000000e+00,  5.53010000e+04,  5.39400000e+03,
        5.97800000e+03, -3.26101761e-03, -3.80467623e-02, -2.20663417e-02,
       -2.06969202e-01, -1.53353333e-01, -3.34080428e-01,  1.87948778e-01,
        1.97183177e-01,  6.31333515e-02, -6.38172477e-02,  3.18448842e-01,
       -1.28581487e-02,  2.54372358e-01,  7.15852305e-02, -4.01395649e-01,
       -2.41744950e-01, -2.24365532e-01, -2.47502491e-01, -7.54199922e-02,
        4.36054915e-01, -1.83131129e-01,  6.35326281e-03,  4.76917207e-01,
        9.54845473e-02, -2.86872275e-02, -4.59304899e-01,  2.59732425e-01,
        3.26311812e-02, -2.70776570e-01,  6.10380173e-02, -5.23494324e-04,
        6.88992143e-02, -1.18377611e-01, -1.30543888e-01,  1.31465495e-01,
        1.58350468e-01, -4.59487131e-03, -3.90655249e-02, -6.79950044e-02,
        7.54482672e-02,  1.24994785e-01, -3.03720534e-01,  7.06110001e-02,
        1.67423427e-01, -2.14514136e-01,  4.95938510e-02, -3.52938883e-02,
       -2.03758657e-01,  

In [ ]:
class Optimus(nn.Module):
    def __init__(self):
        super().__init__()
        self.author_embds = nn.Embedding(1, 20)
        self.cat_embds = nn.Embedding(1, )